# langchain, llm과 친해지기
## 기본적인 사용방법
- OpenAI 라이브러리는 기본적으로 “text-davinci-003” 모델을 이용하고, ChatOpenAI는 “gpt-3.5-turbo”를 이용한다. openai는 ChatOpenAI 사용하는 것을 권장한다. 3.5 모델이 다빈치보다 더 최신모델이고, 가격은 더 저렴하다.
- 주의해야 될 점은 `.env` 파일에 `OPENAI_API_KEY` 라는 이름으로 api키를 이용해야 한다. 같은 이름의 변수를 로드해서 찾기 때문이다.

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

('\n\nThere are 8 planets in the Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

## messages 이용하기
- 구체적으로 질문하고 대답을 받을 수 있다.

In [2]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.1) # temperature 0.1은 덜 창의적으로 대답한다.

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. Any you only reply in Korean."),
    AIMessage(content="안녕하세요. 똥꾸빵꾸입니다."),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

In [4]:
chat.predict_messages(messages)

AIMessage(content='안녕하세요! 저는 똥꾸빵꾸라고 합니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

## prompt templates
- 템플릿은 자주 사용되는 프롬프트를 커스터마이징하면서 재사용할 수 있다.
- PromptTemplate은 스트링 기반, ChatPromptTemplate은 메세지 기반이다.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [6]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. Any you only reply in {language}."),
    ("ai", "안녕하세요. 저의 이름은 {name}."),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Korean",
    name="홍길동",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='안녕하세요. 저의 이름은 홍길동입니다. 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.')

## output parser

- 디비나 다른 곳으로 텍스트를 저장할 때, 기존의 형태가 아닌 다른 형태로 저장하고 싶을 때 사용한다.

In [7]:
from langchain.schema import BaseOutputParser

# 공백을 지우고, 콤마를 기준으로 잘라서 리스트에 담는다.
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("hello, how, are, you")

['hello', 'how', 'are', 'you']

In [13]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do not reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

## lcel
- Langchain expression language
- 랭체인은 내부적으로 template.format_messages를 호출하고, chat.predict를 호출하고, parse를 호출한다.
- 사용설명서 : https://python.langchain.com/docs/expression_language/interface

In [15]:
# 템플릿, 모델, 파서를 체인으로 묶는다.
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

## chaining chains
- all = 체인1 | 체인2 | 아웃풋
- 체인끼리 묶는 것 역시 가능하다.

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 스트리밍과 콜백을 지정해주면 진행상황을 알 수 있다.
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cusine} food.")
])

chef_chain = chef_prompt | chat

In [22]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

In [26]:
final_chain = {"recipe": chef_chain} | veg_chain  # runnable map

final_chain.invoke({
    "cusine": "indian"
})

"""
    1. final_chain = {"recipe": chef_chain.invoke({"cusine: "indian"})} | veg_chain
    2. final_chain = veg_chain.invoke({"recipe": result from step number one})
"""

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's an easy-to-follow recipe:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook u

AIMessage(content="Great choice! Butter Chicken is a delicious Indian dish that can easily be made vegetarian. Here's a modified version of the recipe using alternative ingredients:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan option)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup coconut cream (or cashew cream for a nutty flavor)\n- Salt to taste\n- Fresh cilantro leaves for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to t